# Web Scraping seach_village

# search_village_all_province

In [21]:
from urllib.parse import quote
from gazpacho import get, Soup
import pandas as pd

# Fetch the main page HTML
main_url = "https://healthgate.hss.moph.go.th/search-village"
main_html = get(main_url)
main_soup = Soup(main_html)

# Find the <select> element for provinces
select_element = main_soup.find('select', {'id': 'search_province'})

# Convert the select_element back to HTML string if necessary or directly work with it
select_html = str(select_element)

# `soup` is the parsed HTML of the main page where provinces are listed
select = Soup(select_html)  # Parse the select HTML segment containing provinces
options = select.find('option', mode='all')[1:]  # Skip the first option which is usually a placeholder

# List to hold data from all provinces
all_data = []

for option in options:
    province_name = option.text
    province_value = quote(province_name)  # Ensure the province name is URL-encoded

    # Dynamically build the URL for each province
    url = f"https://healthgate.hss.moph.go.th/search/village?search_province={province_value}"
    
    # Fetch the HTML content for each province
    html = get(url)
    soup = Soup(html)

    # Repeat the extraction process for each province
    table_body = soup.find('tbody')
    rows = table_body.find('tr', mode='all')

    for row in rows:
        cells = row.find('td', mode='all')
        row_data = [cell.text for cell in cells]
        all_data.append(row_data)

# Convert all collected data into a DataFrame
df = pd.DataFrame(all_data, columns=["จังหวัด", "อำเภอ", "ตำบล", "ชื่อหมู่บ้าน", "Login Code", "รหัส รพ.สต"])

In [22]:
df

,จังหวัด,อำเภอ,ตำบล,ชื่อหมู่บ้าน,Login Code,รหัส รพ.สต
0,สมุทรปราการ,เมืองสมุทรปราการ,สำโรงเหนือ,ลิขิต,11010201,
1,สมุทรปราการ,เมืองสมุทรปราการ,สำโรงเหนือ,ทหารเรือ,11010202,
2,สมุทรปราการ,เมืองสมุทรปราการ,สำโรงเหนือ,หัวป่า,11010203,000093300
3,สมุทรปราการ,เมืองสมุทรปราการ,สำโรงเหนือ,ใหญ่,11010204,000093300
4,สมุทรปราการ,เมืองสมุทรปราการ,สำโรงเหนือ,คลองกระทุ่ม,11010205,
...,...,...,...,...,...,...
76543,บึงกาฬ,บุ่งคล้า,โคกกว้าง,โนนไพศาล,38080305,000489500
76544,บึงกาฬ,บุ่งคล้า,โคกกว้าง,ดอนใหญ่,38080306,000489500
76545,บึงกาฬ,บุ่งคล้า,โคกกว้าง,ท่าศิริขันธ์,38080307,000489500
76546,บึงกาฬ,บุ่งคล้า,โคกกว้าง,สมพร,38080308,000489500


In [23]:
# Ensure the 'Login Code' column is treated as a string
df['Login Code'] = df['Login Code'].astype(str)

# Enforce 'Login Code' to have a length of 8 characters
df['Login Code'] = df['Login Code'].apply(lambda x: x.zfill(8) if len(x) < 8 else x[-8:])


In [24]:
df

,จังหวัด,อำเภอ,ตำบล,ชื่อหมู่บ้าน,Login Code,รหัส รพ.สต
0,สมุทรปราการ,เมืองสมุทรปราการ,สำโรงเหนือ,ลิขิต,11010201,
1,สมุทรปราการ,เมืองสมุทรปราการ,สำโรงเหนือ,ทหารเรือ,11010202,
2,สมุทรปราการ,เมืองสมุทรปราการ,สำโรงเหนือ,หัวป่า,11010203,000093300
3,สมุทรปราการ,เมืองสมุทรปราการ,สำโรงเหนือ,ใหญ่,11010204,000093300
4,สมุทรปราการ,เมืองสมุทรปราการ,สำโรงเหนือ,คลองกระทุ่ม,11010205,
...,...,...,...,...,...,...
76543,บึงกาฬ,บุ่งคล้า,โคกกว้าง,โนนไพศาล,38080305,000489500
76544,บึงกาฬ,บุ่งคล้า,โคกกว้าง,ดอนใหญ่,38080306,000489500
76545,บึงกาฬ,บุ่งคล้า,โคกกว้าง,ท่าศิริขันธ์,38080307,000489500
76546,บึงกาฬ,บุ่งคล้า,โคกกว้าง,สมพร,38080308,000489500


In [28]:
# Export the DataFrame to a CSV file
df.to_csv('search_village.csv', index=False)